In [15]:
import geopandas as gpd
import pandas as pd
from sklearn.cluster import DBSCAN
import folium
import branca.colormap as cm  # Add this import
from folium.plugins import HeatMap

In [2]:
# Import flats with population
flats_with_pop = gpd.read_file('/home/silas/projects/msc_thesis/data/derived_data/flats_population.gpkg')

# Import isochrones
merged_isochrones_gdf= gpd.read_file('/home/silas/projects/msc_thesis/data/derived_data/isochrones_1-10min.gpkg')
# Load or create rcps GeoDataFrame
rcps=gpd.read_file('/home/silas/projects/msc_thesis/data/raw_data/geodata_stadt_Zuerich/recycling_sammelstellen/data/stzh.poi_sammelstelle_view.shp')
flats_with_pop.to_crs(epsg=4326, inplace=True)
rcps.to_crs(epsg=4326, inplace=True)

In [3]:
# remove flats with population 0
flats_with_pop = flats_with_pop[flats_with_pop['est_pop'] > 0]


# Step 1: Verify 'time' column exists in merged_isochrones_gdf
if 'time' not in merged_isochrones_gdf.columns:
    raise KeyError("'time' column is missing in merged_isochrones_gdf")

# Step 2: Spatial join with a left join to retain all flats
joined = gpd.sjoin(flats_with_pop, merged_isochrones_gdf[['geometry', 'time']], how='left', predicate='within')

# Step 3: Assign a default high time value to unserved flats
iso_threshold = 10
joined['time'] = joined['time'].fillna(iso_threshold + 1)  # Assign a value greater than threshold

# Step 4: Get only the shortest time for each flat
joined = joined.groupby('egid').agg({
    'est_pop': 'first',
    'geometry': 'first',
    'time': 'min'
}).reset_index()

# Step 5: Filter unserved flats with time greater than 5 minutes
high_pop_unserved = joined[joined['time'] > 10]
high_pop_unserved = gpd.GeoDataFrame(high_pop_unserved, geometry='geometry', crs="EPSG:4326")

# export to file
high_pop_unserved.to_file('/home/silas/projects/msc_thesis/data/derived_data/high_pop_unserved.gpkg', driver='GPKG')


In [4]:
high_pop_unserved = gpd.GeoDataFrame(high_pop_unserved, geometry='geometry', crs="EPSG:4326")

# Ensure the centroid calculation does not raise a warning
high_pop_unserved = high_pop_unserved.set_geometry(high_pop_unserved.geometry.centroid)

definedcoords = high_pop_unserved.geometry
X = pd.DataFrame({
    'x': definedcoords.x,
    'y': definedcoords.y,
    'population': high_pop_unserved['est_pop']
})

# Step 3: Apply DBSCAN clustering
db = DBSCAN(eps=0.005, min_samples=10).fit(X[['x', 'y']])
X['cluster'] = db.labels_

# Remove noise points
clusters = X[X['cluster'] != -1]

# Step 4: Calculate cluster centers weighted by population
cluster_centers = clusters.groupby('cluster').apply(
    lambda df: pd.Series({
        'x': (df['x'] * df['population']).sum() / df['population'].sum(),
        'y': (df['y'] * df['population']).sum() / df['population'].sum()
    })
).reset_index()

# Step 5: Create GeoDataFrame for new collection points
new_points = gpd.GeoDataFrame(
    cluster_centers,
    geometry=gpd.points_from_xy(cluster_centers['x'], cluster_centers['y']),
    crs="EPSG:4326"
)

/tmp/ipykernel_6493/3337235828.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  high_pop_unserved = high_pop_unserved.set_geometry(high_pop_unserved.geometry.centroid)
/tmp/ipykernel_6493/3337235828.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cluster_centers = clusters.groupby('cluster').apply(


In [5]:


# Load or create rcps GeoDataFrame
# Assuming rcps is a GeoDataFrame containing existing collection points

# Step 6: Plotting
m = folium.Map(location=[47.3769, 8.5417], zoom_start=13)

# Add existing collection points
""" for _, row in rcps.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row['adresse'],
        icon=folium.Icon(color='green', icon='recycle', prefix='fa')
    ).add_to(m) """

#add high population unserved flats
for _, row in high_pop_unserved.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        popup=f"Flat ID: {row['egid']}<br>Duration: {row['time']:.2f} min, Population: {row['est_pop']}",
        color='red',
        radius=5,
        fill=True

    ).add_to(m)


# Add new collection points
for _, row in new_points.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup='New Collection Point',
        icon=folium.Icon(color='blue', icon='plus', prefix='fa')
    ).add_to(m)
m.save('/home/silas/projects/msc_thesis/data/derived_data/new_collection_points_dbscan.html')
m

In [9]:
import openrouteservice
from shapely.geometry import shape
import os

# get ORS key
ors_key = os.getenv('ORS_API_KEY')

# Initialize ORS client
client = openrouteservice.Client(key=ors_key)

def generate_isochrones(client, locations, time_limit):
    params = {
        "locations": [locations],
        "range": [time_limit],
        "range_type": "time",
        "location_type": "start",
        "smoothing": 0.3,
        "profile": "foot-walking",
    }
    isochrones = client.isochrones(**params)
    return isochrones

# Generate isochrones for new points
new_isochrones = []
time_limits = [60, 120, 180, 240, 300, 360, 420, 480, 540, 600]  # 10 minutes

for time_limit in time_limits:
    for _, row in new_points.iterrows():
        lon, lat = row.geometry.x, row.geometry.y
        isochrone = generate_isochrones(client, [lon, lat], time_limit)
        if isochrone:
            for feature in isochrone['features']:
                new_isochrones.append({
                    'geometry': shape(feature['geometry']),
                    'time': time_limit / 60,
                    'rcp_id': f"new_{row['cluster']}"
                })

# Create GeoDataFrame for new isochrones
new_isochrones_gdf = gpd.GeoDataFrame(new_isochrones, crs="EPSG:4326")

# Merge new isochrones with existing isochrones
merged_isochrones_gdf = pd.concat([merged_isochrones_gdf, new_isochrones_gdf], ignore_index=True)

# Save the updated isochrones to file
merged_isochrones_gdf.to_file('/home/silas/projects/msc_thesis/data/derived_data/updated_isochrones.gpkg', driver='GPKG')

/home/silas/miniconda3/envs/geo_env/lib/python3.13/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/silas/miniconda3/envs/geo_env/lib/python3.13/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/silas/miniconda3/envs/geo_env/lib/python3.13/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/silas/miniconda3/envs/geo_env/lib/python3.13/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_c

In [10]:
# Create a folium map centered around Zurich
m = folium.Map(location=[47.3769, 8.5417], zoom_start=13)

# Add new isochrones to the map
for _, row in new_isochrones_gdf.iterrows():
    folium.GeoJson(
        row['geometry'],
        name=f"Isochrone {row['rcp_id']} - {row['time']} min",
        style_function=lambda x: {'color': 'blue', 'weight': 2, 'fillOpacity': 0.2}
    ).add_to(m)

# Add layer control to toggle isochrones
folium.LayerControl().add_to(m)

# Display the map
m

In [11]:

# Function to prepare heatmap data

merged_isochrones_gdf.to_crs(epsg=4326, inplace=True)
def prepare_heatmap_data(merged_isochrones_gdf):
    """
    Prepare heatmap data from merged isochrones.

    Parameters:
    - merged_isochrones_gdf: GeoDataFrame with merged isochrones.

    Returns:
    - List of [latitude, longitude, intensity] for HeatMap.
    """
    heat_data = []
    for _, row in merged_isochrones_gdf.iterrows():
        if 'time' not in row:
            print("Missing 'time' column in merged_isochrones_gdf")
            continue
        centroid = row.geometry.centroid
        lat, lon = centroid.y, centroid.x
        intensity = row['time'] / 60  # Convert time to minutes for intensity
        heat_data.append([lat, lon, intensity])
    return heat_data


In [ ]:

from folium.plugins import MarkerCluster

heat_data = prepare_heatmap_data(merged_isochrones_gdf)

# Initialize Folium map centered around Zurich
m = folium.Map(location=[47.3769, 8.5417], zoom_start=13)

# Create a continuous color map for the heatmap
colormap = cm.linear.YlOrRd_09.scale(0, 10)
colormap.caption = 'Isochrone Time (minutes)'
colormap.add_to(m)

# Add merged isochrones to the map
for _, row in merged_isochrones_gdf.iterrows():
    folium.GeoJson(
        row['geometry'],
        name=f"Isochrone {row['time']} min",
        style_function=lambda feature, time=row['time']: {
            'fillColor': colormap(time),
            'color': colormap(time),
            'weight': 1,
            'fillOpacity': 0.9,
        }
    ).add_to(m)

# Add recycling collection points to the map using MarkerCluster
marker_cluster = MarkerCluster().add_to(m)
for _, row in rcps.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row['adresse'],
        icon=folium.Icon(color='green', icon='recycle', prefix='fa')
    ).add_to(marker_cluster)

# Save and display the map
heatmap_path = '/home/silas/projects/msc_thesis/data/derived_data/heatmap_test_after_cluster.html'
m.save(heatmap_path)
m